# Evaluation of different Parameters (Features: Section Means)

In [1]:
import pandas as pd
import numpy as np
from pivottablejs import pivot_ui
import sys
sys.path.append('..')  # in order to import modules from my own package

# my package
from packageMeinhart import PhysioDataHandler as PDH
from packageMeinhart.functionsMasterProjectMeinhart import print_precision_recall_accuracy
from packageMeinhart.functionsMasterProjectMeinhart import print_misclassified_data_points

## Using the class *PhysioData_SectionFeatures* for feature generation

In [2]:
help(PDH.PhysioData_SectionFeatures)

Help on class PhysioData_SectionFeatures in module packageMeinhart.PhysioDataHandler:

class PhysioData_SectionFeatures(builtins.object)
 |  Class for feature generation using section means.
 |  There are various selectable options --> see Parameters. 
 |  
 |  Parameters
 |  ----------
 |  num_sections : int
 |      Number of equally partitioned sections to split the single repetitions of the signals.
 |      
 |  test_subject_ids : int or list (of int)
 |      Subject IDs to select for testing (e.g. [1, 2, 3]).
 |      --> default -1: Select all subjects.
 |      --> if test_subject_ids is an empty list: empty DataFrame is returned by corresponding method.
 |      
 |  train_subject_ids : int or list
 |      Subject IDs to select for training (e.g. [1, 2, 3]).
 |      --> default -1: Select all subjects not in test_subject_ids.
 |      --> if train_subject_ids is an empty list: empty DataFrame is returned by corresponding method.
 |      
 |  test_rep_nums : int or list
 |      Repet

### Create instance of physio data class

In [3]:
PD1 = PDH.PhysioData_SectionFeatures(num_sections=10,
                                     test_subject_ids=1,
                                     train_subject_ids=-1,
                                     test_rep_nums=-1,
                                     train_rep_nums=-1,
                                     test_ex_abbrs=-1,
                                     train_ex_abbrs=-1,
                                     with_non_Ex=True,
                                     rot_axis_test_data=0,
                                     rot_angle_test_data=0,
                                     add_noise_test_data=False,
                                     add_noise_train_data=False,
                                     snr_db=20,
                                     csv_data_dir='E:\Physio_Data_Split_Ex_and_NonEx',
                                     csv_skiprows=0,
                                     csv_separator=',',
                                     data_base_path='E:\Physio_Data\DataBase_Physio_with_nonEx.db',
                                     print_progress=True,
                                     signal_abbrs=['Acc','Gyr'],
                                     signal_orientations=['x','y','z'],
                                     labels_abbr2num_dict={'RF':0,'RO':1,'RS':2,'LR':3,'BC':4,'TC':5,
                                                           'MP':6,'SA':7,'P1':8,'P2':9,'NE':10},
                                     sub_id_key='subject_id',
                                     num_rep_key='num_rep',
                                     abbreviation_key='abbreviation',
                                     start_time_key='start_time',
                                     stop_time_key='stop_time',
                                     csv_file_key='csv_file',
                                     sampling_rate=256)

### Inspecting selected data for testing

In [4]:
pivot_ui(PD1.get_test_data_points(), 
         rows=['abbreviation'], 
         cols=['subject_id', 'num_rep'], 
         outfile_path="PD1_test.html")

### Inspecting selected data for training

In [5]:
pivot_ui(PD1.get_train_data_points(), 
         rows=['abbreviation'], 
         cols=['subject_id', 'num_rep'], 
         outfile_path="PD1_train.html")

## Classification (ML part)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
# StandardScaler raises the following warning:
# --> DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
# if we want to ignore that:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

### Try it with different ML models

In [7]:
# create ML model
ML_model = RandomForestClassifier(n_estimators=50, max_leaf_nodes=40, n_jobs=-1, random_state=42)
#ML_model = make_pipeline(StandardScaler(), SVC(random_state=42)) # Support Vector Classifier with input scaling

# train the model
ML_model.fit(PD1.X_train(), PD1.y_train())

# predict labels
y_pred = ML_model.predict(PD1.X_test())

# show results
print('Model: ' + type(ML_model).__name__ + '\n')
print('Total Accuracy: {:.2f}%\n'.format((accuracy_score(PD1.y_test(), y_pred))*100))
print_precision_recall_accuracy(y_pred, PD1.y_test())
print('')
print_misclassified_data_points(y_pred, PD1.y_test())

Model: RandomForestClassifier

Total Accuracy: 98.87%

Exercise	Precision [%]	Recall [%]	Accuracy [%]
  RF		  100.00	   93.33	   99.72
  RO		   93.75	  100.00	   99.72
  RS		  100.00	  100.00	  100.00
  LR		  100.00	  100.00	  100.00
  BC		  100.00	  100.00	  100.00
  TC		  100.00	  100.00	  100.00
  MP		   85.71	  100.00	   99.29
  SA		  100.00	  100.00	  100.00
  P1		  100.00	   96.67	   99.86
  P2		  100.00	  100.00	  100.00
  NE		   99.75	   98.77	   99.15

8 misclassified (709 test data points):
RF classified as RO
RF classified as RO
P1 classified as NE
NE classified as MP
NE classified as MP
NE classified as MP
NE classified as MP
NE classified as MP


## Grid Search and Cross Validation

In [14]:
pipe_elements = [('scale', StandardScaler()), ('clf', SVC())]
#pipe_elements = [('scale', StandardScaler()), ('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(pipe_elements)

param_grid = {'clf__C': [0.1, 1, 10],
              'clf__gamma': ['scale', 0.01, 0.1]}

# include PCA
#param_grid = {'reduce_dim__n_components': [10, 20, 30],
#              'clf__C': [1, 10, 100],
#              'clf__gamma': [1, 10, 100]}

# C: Penalty parameter C of the error term. (Regularisation parameter)
# gamma: Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. (Bandwidth of kernel)

# splitting strategy for grid search: stratified CV with 5 folds
grid_search = GridSearchCV(pipe, 
                           param_grid=param_grid, 
                           cv=5, scoring='accuracy', 
                           verbose=10, 
                           n_jobs=-1, 
                           return_train_score=True)

In [15]:
# apply grid search and cross validation
grid_search.fit(PD1.X_train(), PD1.y_train())

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed:  1.1min remaining:    3.1s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__C': [0.1, 1, 10], 'clf__gamma': ['scale', 0.01, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=10)

In [16]:
# show score (test data)
print('Accuracy: {:.2f}%'.format(grid_search.score(PD1.X_test(), PD1.y_test())*100))

Accuracy: 98.73%


In [17]:
# show best parameters
print('Best parameters: {}'.format(grid_search.best_params_))

Best parameters: {'clf__C': 1, 'clf__gamma': 'scale'}


In [18]:
# show best score of cross validation
print('Best score at cross validatoin: {:.2f}%'.format(grid_search.best_score_*100))

Best score at cross validatoin: 98.95%


In [19]:
# show best estimator
print('Best estimator: {}'.format(grid_search.best_estimator_))

Best estimator: Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])


In [20]:
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_clf__C,param_clf__gamma,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,1.849506,0.466027,0.977288,0.979167,0.1,scale,"{'clf__C': 0.1, 'clf__gamma': 'scale'}",5,0.978013,0.981603,...,0.982829,0.978763,0.971358,0.976929,0.985246,0.978776,0.092449,0.024683,0.006289,0.001524
1,1.956912,0.724841,0.955392,0.957676,0.1,0.01,"{'clf__C': 0.1, 'clf__gamma': 0.01}",8,0.945440,0.956051,...,0.953393,0.960588,0.943535,0.947121,0.986885,0.975714,0.278778,0.256786,0.016059,0.010245
2,10.369793,0.678239,0.901961,0.934069,0.1,0.1,"{'clf__C': 0.1, 'clf__gamma': 0.1}",9,0.909609,0.933974,...,0.921504,0.932203,0.893617,0.930176,0.876230,0.936939,0.559981,0.021199,0.015592,0.002675
3,0.672039,0.191011,0.989542,0.991748,1,scale,"{'clf__C': 1, 'clf__gamma': 'scale'}",1,0.985342,0.992845,...,0.995094,0.990811,0.988543,0.991833,0.988525,0.992449,0.080828,0.012602,0.003193,0.000834
4,0.503829,0.168010,0.987908,0.990849,1,0.01,"{'clf__C': 1, 'clf__gamma': 0.01}",3,0.984528,0.991619,...,0.994276,0.990402,0.985270,0.991017,0.988525,0.991837,0.029480,0.006293,0.003470,0.000892
5,10.578405,0.489828,0.965523,0.995180,1,0.1,"{'clf__C': 1, 'clf__gamma': 0.1}",6,0.967427,0.996934,...,0.973017,0.994895,0.970540,0.994079,0.945082,0.995306,0.429613,0.027171,0.010362,0.000962
6,0.660238,0.160809,0.987092,0.994567,10,scale,"{'clf__C': 10, 'clf__gamma': 'scale'}",4,0.986971,0.995707,...,0.988553,0.994282,0.987725,0.993671,0.979508,0.995306,0.079750,0.012140,0.004266,0.000802
7,0.441025,0.116407,0.988072,0.993750,10,0.01,"{'clf__C': 10, 'clf__gamma': 0.01}",2,0.988599,0.994890,...,0.987735,0.992649,0.988543,0.993467,0.982787,0.994898,0.015850,0.002728,0.003150,0.000972
8,10.438597,0.471627,0.961765,0.999591,10,0.1,"{'clf__C': 10, 'clf__gamma': 0.1}",7,0.963355,0.999387,...,0.967294,1.000000,0.968903,1.000000,0.938525,0.999184,0.484676,0.109767,0.011846,0.000342


## Set parameters and evaluate directly

In [28]:
PD2 = PDH.PhysioData_SectionFeatures(num_sections=10,
                                     test_subject_ids=1,
                                     train_subject_ids=[2,3],
                                     test_rep_nums=-1,
                                     train_rep_nums=[10, 15],
                                     test_ex_abbrs=['RF','RO','RS','LR'],
                                     train_ex_abbrs=['RF','RO','RS','LR'],
                                     with_non_Ex=True,
                                     rot_axis_test_data=[0,1],
                                     rot_angle_test_data=[10,5],
                                     add_noise_test_data=True,
                                     add_noise_train_data=False,
                                     snr_db=20)

# create ML model
ML_model = RandomForestClassifier(n_estimators=50, max_leaf_nodes=40, n_jobs=-1, random_state=42)
#ML_model = make_pipeline(StandardScaler(), SVC(random_state=42)) # Support Vector Classifier with input scaling

# train the model
ML_model.fit(PD2.X_train(), PD2.y_train())

# predict labels
y_pred = ML_model.predict(PD2.X_test())

# show results
print('Model: ' + type(ML_model).__name__ + '\n')

print('Total Accuracy: {:.2f}%\n'.format((accuracy_score(PD2.y_test(), y_pred))*100))
print_precision_recall_accuracy(y_pred, PD2.y_test())

report = classification_report(PD2.y_test(), y_pred, 
                               labels=np.arange(0,11),
                               target_names=['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2','NE'],
                               sample_weight=None, output_dict=True)

report_df = pd.DataFrame.from_dict(report, orient='index')
display(report_df)

print('')
print_misclassified_data_points(y_pred, PD2.y_test())

Model: RandomForestClassifier

Total Accuracy: 95.00%

Exercise	Precision [%]	Recall [%]	Accuracy [%]
  RF		  100.00	   80.00	   95.00
  RO		   83.33	  100.00	   95.00
  RS		  100.00	  100.00	  100.00
  LR		  100.00	  100.00	  100.00
  BC		     nan	     nan	  100.00
  TC		     nan	     nan	  100.00
  MP		     nan	     nan	  100.00
  SA		     nan	     nan	  100.00
  P1		     nan	     nan	  100.00
  P2		     nan	     nan	  100.00
  NE		     nan	     nan	  100.00


..\packageMeinhart\functionsMasterProjectMeinhart.py:895: RuntimeWarning: invalid value encountered in long_scalars
  precision = TP / (TP+FP)
..\packageMeinhart\functionsMasterProjectMeinhart.py:896: RuntimeWarning: invalid value encountered in long_scalars
  recall = TP / (TP+FN)
E:\Programme\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\Programme\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,precision,recall,f1-score,support
BC,0.000000,0.000000,0.000000,0
LR,1.000000,1.000000,1.000000,30
MP,0.000000,0.000000,0.000000,0
NE,0.000000,0.000000,0.000000,0
P1,0.000000,0.000000,0.000000,0
P2,0.000000,0.000000,0.000000,0
RF,1.000000,0.800000,0.888889,30
RO,0.833333,1.000000,0.909091,30
RS,1.000000,1.000000,1.000000,30
SA,0.000000,0.000000,0.000000,0



6 misclassified (120 test data points):
RF classified as RO
RF classified as RO
RF classified as RO
RF classified as RO
RF classified as RO
RF classified as RO


In [12]:
report = classification_report(PD2.y_test(), y_pred, 
                               labels=np.arange(0,11),
                               target_names=['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2','NE'],
                               sample_weight=None, output_dict=True)

report_df = pd.DataFrame.from_dict(report, orient='index')
display(report_df)

E:\Programme\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,precision,recall,f1-score,support
BC,1.000000,1.000000,1.000000,5
LR,0.000000,0.000000,0.000000,5
MP,0.714286,1.000000,0.833333,5
NE,0.983010,0.995086,0.989011,407
P1,1.000000,1.000000,1.000000,5
P2,1.000000,1.000000,1.000000,5
RF,1.000000,1.000000,1.000000,5
RO,1.000000,0.600000,0.750000,5
RS,1.000000,1.000000,1.000000,5
SA,1.000000,1.000000,1.000000,5


In [10]:
pipe

NameError: name 'pipe' is not defined

In [ ]:
pipe.steps[0]  

In [ ]:
pipe.named_steps['clf'] 

In [ ]:

## set number of principal components
#number_principal_comp = 30
#
## make pca model
#pca = PCA(n_components=number_principal_comp)
#
## create new features from PCA projections
#X_train_pca = pca.fit_transform(X_train_for_pca)
#X_test_pca = pca.transform(X_test_for_pca)
#
#
## make LDA model
#lda = LDA()
## create new features from LDA projections
#X_train_lda = lda.fit_transform(X_train_for_lda, y_train_lda)
#X_test_lda = lda.transform(X_test_for_lda)


$accuracy = \frac{TP + TN}{TP + TN + FP + FN}$

$precision = \frac{TP}{TP + FP}$

$recall = \frac{TP}{TP + FN}$

$f{\text-}score = \frac{precision \cdot recall}{precision + recall}$